In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from functools import reduce
import operator

import pymongo

In [2]:
load_dotenv()

True

#### Paris: Palais Garnier Sao Paulo: Sala Sao Paulo

In [37]:
paris_ld = {'type': 'Point', 'coordinates': [48.87241163769417,2.3317031342524017]}

In [23]:
melbourne_cbd = {'type': 'Point', 'coordinates': ['-37.81227175879301','144.96262186884945']}

In [24]:
sao_paulo_zc = {'type': 'Point', 'coordinates': ['-23.533215960575276','-46.639408758375076']}

In [26]:
def getdata_fsq(url, city, venue):
    client_id = os.getenv("tok1")
    client_secret = os.getenv("tok2")
    url_ = f'{url}'
    c = city['coordinates'][0]
    c1 = city['coordinates'][1]
    parametros = {"client_id": client_id, "client_secret": client_secret, "v": "20180323", "ll": f'{c}, {c1}',
                  "query": venue, "limit": 55}
    resp = requests.get(url_, params=parametros).json()
    return resp

#### HACER UNA FUNCION PARA GENERAR DATAS POR CADA RUBRO, POR CADA CIUDAD

In [ ]:
#def func(url, city, venue):
    cathegories = []
    for c in cathegories:
        

In [27]:
datap = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'vegan')

In [380]:
datapa = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'airport')

In [50]:
datapg = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'gare')

In [38]:
datapsb = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'starbucks')

In [56]:
datapv = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'veterinaire')

In [60]:
datapb = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'basketball')

In [65]:
datasc = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'ecole')

In [381]:
datapa_= datapa['response']['venues']

In [391]:
datapa_[0]['location']['city']

'Paris'

In [384]:
def getFromDict(dic,map_):
    return reduce(operator.getitem, map_, dic)

In [385]:
def type_point(gps):
    return {"type": "Point", "coordinates": gps}

#### HACER FUNCION PARA GENERAR UN JSON POR CADA DATA agregar Tipo (vegan, starb, etc)

In [ ]:
def jsongenerator(data):
    d = data
    map_name = ['name']
    map_categories = ['categories']
    map_lat = ['location', 'lat']
    map_long = ['location', 'lng']
    l = []
    for dic in d:
        ndic = {}
        ndic['name']= getFromDict(dic,map_name)
        ndic['categories']= getFromDict(dic,map_categories)
        ndic['latitude']= getFromDict(dic,map_lat)
        ndic['longitude']= getFromDict(dic,map_long)
        ndic['location']= type_point([ndic["latitude"],ndic["longitude"]])
        l.append(ndic)
    return l

def jsonxport(name, l):
    n = f'{name}.json'
    with open(n, 'w') as f:
        json.dump(l, f)

In [389]:
map_name = ['name']
map_categories = ['categories']
#map_city = ['location', 'city']
map_lat = ['location', 'lat']
map_long = ['location', 'lng']
gastr = []
for dic in datapa_:
    ndic = {}
    ndic['name']= getFromDict(dic,map_name)
    ndic['categories']= getFromDict(dic,map_categories)
  #  ndic['city']= getFromDict(dic,map_city)
    ndic['latitude']= getFromDict(dic,map_lat)
    ndic['longitude']= getFromDict(dic,map_long)
    ndic['location']= type_point([ndic["latitude"],ndic["longitude"]])
    gastr.append(ndic)

In [388]:
map_name = ['name']
map_city = ['location', 'city']
map_lat = ['location', 'lat']
map_long = ['location', 'lng']
gastr = []
for dic in datapsb_:
    ndic = {}
    ndic['name']= getFromDict(dic,map_name)
    ndic['city']= getFromDict(dic,map_city)
    ndic['latitude']= getFromDict(dic,map_lat)
    ndic['longitude']= getFromDict(dic,map_long)
    ndic['location']= type_point([ndic["latitude"],ndic["longitude"]])
    gastr.append(ndic)

#### HACER FUNCION PARA GENERAR LOS JSON

In [ ]:
def jsongenerator(name):
    n = f'{name}.json'
    with open(n, 'w') as f:
    json.dump(gastr, f)

In [115]:
with open('vegans.json', 'w') as f:
    json.dump(gastr, f)

In [76]:
with open('starbucks.json', 'w') as f:
    json.dump(gastr, f)

In [392]:
with open('airport.json', 'w') as f:
    json.dump(gastr, f)

#### llamada a la collection

In [127]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("GeoSpatialProyect")

In [128]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoSpatialProyect')

In [393]:
g = db.get_collection("airport")

In [394]:
g.find_one().keys()

dict_keys(['_id', 'name', 'categories', 'latitude', 'longitude', 'location'])

In [ ]:
#cond1 = {'categories.name': 'Vegetarian / Vegan Restaurant'}

In [420]:
cond1 = {'categories.name': 'Bus Stop'}

In [427]:
cond2 = {"location": {"$near": {"$geometry": paris_ld, "$minDistance": 0 ,"$maxDistance": 2000}}}

In [428]:
query2 = {'$and': [cond1, cond2]}

In [429]:
list(g.find(query2))

[{'_id': ObjectId('6133f8adb66677fc14d1139a'),
  'name': 'Arrêt Paris-Opéra [Roissybus]',
  'categories': [{'id': '52f2ab2ebcbc57f1066b8b4f',
    'name': 'Bus Stop',
    'pluralName': 'Bus Stops',
    'shortName': 'Bus Stop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/busstation_',
     'suffix': '.png'},
    'primary': True}],
  'latitude': 48.87208680097662,
  'longitude': 2.330820463639864,
  'location': {'type': 'Point',
   'coordinates': [48.87208680097662, 2.330820463639864]}}]

In [283]:
#q1 = db.gastronomy.aggregate([{'$geoNear': {'near': paris_ld, 'distanceField': "distance", 'maxDistance': 1000, 'distanceMultiplier': 6371,
#                                   'spherical': True}}])